# Tutorial 3: Initializing a conditioning random field chain

### First, obtaining information we need as in T1 and T2

In [1]:
# load compiled bed elevation measurements
df = pd.read_csv('')

In [ ]:
# create a grid of x and y coordinates
x_uniq = np.unique(df.X)
y_uniq = np.unique(df.Y)

xmin = np.min(x_uniq)
xmax = np.max(x_uniq)
ymin = np.min(y_uniq)
ymax = np.max(y_uniq)

cols = len(x_uniq)
rows = len(y_uniq)

resolution = 1000

xx, yy = np.meshgrid(x_uniq, y_uniq)

In [ ]:
# load other data
velx, vely, velxerr, velyerr, fig = Topography.load_vel_measures('../../Data/antarctica_ice_velocity_450m_v2.nc', xx, yy)
dhdt, fig = Topography.load_dhdt('../Data/ANT_G1920_GroundedIceHeight_v01.nc',xx,yy,interp_method='linear',begin_year = 2013,end_year=2015,month=7)
smb, fig = Topography.load_smb_racmo('../Data/SMB_RACMO2.3p2_yearly_ANT27_1979_2016.nc', xx, yy, interp_method='spline',time=2014)
bm_mask, bm_source, bm_bed, bm_surface, bm_errbed, fig = Topography.load_bedmachine('../../Data/BedMachineAntarctica-v3.nc', xx, yy)

In [ ]:
# find variograms
df_bed = df.copy()
df_bed = df_bed[df_bed["bed"].isnull() == False]
data = df_bed['bed'].values.reshape(-1,1)
coords = df_bed[['X','Y']].values
roughness_region_mask = (df_bed['bedmachine_mask'].values)==2 # Read BedMachine user guide for the meaning of values == 2 https://nsidc.org/data/nsidc-0756/versions/3

nst_trans, Nbed_radar, varios, fig = MCMC.fit_variogram(data, coords, roughness_region_mask, maxlag=70000, n_lags=70)

In [ ]:
# calculate high velocity region
ocean_mask = (bm_mask == 0) | (bm_mask == 3) # utilize the mask in BedMachine dataset to characterize ice regions
grounded_ice_mask = (bm_mask == 2)
distance_max = 3000
velocity_threshold = 50
highvel_mask = Topography.get_highvel_boundary(velx, vely, velocity_threshold, grounded_ice_mask, ocean_mask, distance_max, xx, yy)

In [ ]:
# load the initial bed
sgs_bed = np.loadtxt('sgs_bed.txt')
thickness = bm_surface - sgs_bed
sgs_bed = np.where((thickness<=0)&(bm_mask==2), bm_surface-1, sgs_bed)

In [ ]:
# initiate conditioning data and a mask of conditioning data
cond_bed = df['bed'].values.reshape(xx.shape)
data_mask = ~np.isnan(cond_bed)

### Then initiating the chain

#### Initiate an object of the class chain_crf

Let's first initiate the chain. Here, we created an object of the class chain_crf. This initialization process requires several input that are essential for later calculations

In [ ]:
crf_1 = MCMC.chain_crf(xx, yy, bed, bm_surface, velx, vely, dhdt, smb, cond_bed, data_mask, grounded_ice_mask, resolution)

Now, the object 'crf_1' have all the properties you have assigned in the initialization process. Try typing *crf_1.xx* or *crf_1.cond_bed*, or any other argument to check them.

In [ ]:
plt.pcolormesh(xx, yy, crf_1.bm_surface)
plt.axis('scaled')
plt.colorbar()

In [ ]:
plt.pcolormesh(xx, yy, crf_1.data_mask)
plt.axis('scaled')
plt.colorbar()

In [ ]:
plt.pcolormesh(xx, yy, crf_1.velx)
plt.axis('scaled')
plt.colorbar()

The second function required, is *set_high_vel_region*. In this function, the first boolean argument decide whether the MCMC update will be inside the high velocity, the second argument specify the exact region of high velocity (where the high_vel_mask == 1)

In [ ]:
ccrf.set_high_vel_region(False,highvel_mask)

We also want to specify how we want to define the loss function used in the chain.

*map_func* determine the distribution of mass conservation residuals, whereas the *diff_func* determine the distribution of the difference between radar measurements and simulated topography. If the residuals has a Gaussian distribution, the corresponding function will be 'sumsquare'. If you do not want to include either of these loss, simple put *map_func = None* or *diff_func = None*. 

*sigma_mc* and *sigma_data* determine the standard deviation of the distribution of mass conservation residual or differences to radar data. *massConvInRegion* and *dataDiffInRegion* specify whether the two losses should be calculated for only inside the high velocity region or not

In [ ]:
ccrf.set_loss_type(map_func='sumsquare', diff_func='sumsquare', sigma_mc=3, sigma_data=80, massConvInRegion=False)

#### Initiate an object of the class RandField

Next step is to set up the parameters of the random field perturbation

In [ ]:
range_max_x = 50e3 #in terms of meters in lateral dimension, regardless of resolution of the map
range_max_y = 50e3
range_min_x = 10e3
range_min_y = 10e3
scale_min = 100 #in terms of meters in vertical dimension, how much you want to multiply the perturbation by
scale_max = 400
nugget_max = 0
random_field_model = 'Exponential' # currently only supporting 'Gaussian' or 'Exponential'
isotropic = True

rf1 = MCMC.RandField(range_max_x, range_max_y, range_min_x, range_min_y, scale_min, scale_max, nugget_max, random_field_model, isotropic)

Then we can set up the size of the blocks used in the update

*set_block_sizes* function create a list of possible block sizes, which will be accessible via rf1.pairs after the function is called.

*set_block_sizes* function also has an optional argument *steps*, it specify how many 'steps' between the min_block and max_block will be divided into. For example, for min_block_x = 20 and max_block_x = 50, steps = 4 will divide the range into [20, 30, 40, 50]. For min_block_y = 10 and max_block_y = 55, it will divide the range into [10, 25, 40, 55]. Then each size on the list for x will pair up with all size on the list for y, creating a list of following

rf1.pairs = [[20,10],[20,25],[20,40],[20,55],[30,10],[30,25],[30,40],[30,55],[40,10],[40,25],[40,40],[40,55],[50,10],[50,25],[50,40],[50,55]]

When randomly deciding the size of the block, one of the size on the list will be chosen and will be used

In [ ]:
min_block_x = 20
max_block_x = 50
min_block_y = 20
max_block_y = 50
rf1.set_block_sizes(min_block_x, max_block_x, min_block_y, max_block_y)

Finally, if you wish to use conditional block update, then the RandField object requires specifying the logistic function used to calculate the conditioning weight of the random field

The logistic function is used to calculate the weight for conditioning to the edge of the block and the weight for conditioning to radar measurements. When updating the field, the weight will be 1 at conditioning data and block edges, and it logistically decays to 0 at location *max_dist* away from any conditioning data

In [ ]:
logis_func_L = 2
logis_func_x0 = 0
logis_func_k = 6
logis_func_offset = 1
max_dist = varios[2][0] # set to the distance between two points on the map where the correlation vanish / is minimal

rf1.set_block_param(logis_func_L, logis_func_x0, logis_func_k, logis_func_offset, max_dist, resolution)

#### Run the Markov chain

At last, these information need to be known to the crf chain. The function *set_crf_data_weight* calculate the weight for conditioning to the radar measurements

In [ ]:
ccrf.set_crf_data_weight(rf1)
ccrf.set_update_type('RF')

And then we can start the chain by specifying how many iterations it should go through

In [ ]:
seed = 0
randomGenerator = np.random.default_rng(seed)

bed_cache, loss_mc_cache, loss_data_cache, loss_cache, step_cache, resampled_times, blocks_cache = ccrf.run(n_iter=5000, RF=rf1, rng=randomGenerator)